In [14]:
from HierarchicalTransformer import * #这句需要运行一下，加载需要调用的库

In [15]:
#加载数据集，主要修改domain_ID,对应测试不同目标域的效果,从0,1,3,4中进行选择,2可以先不测

In [16]:
    data_dir1 = r"../../autodl-tmp/data/pheme-rnr-dataset/"
    
#     os.environ['CUDA_VISIBLE_DEVICES'] = "0" 

    events_list = ['charliehebdo', 'ferguson', 'germanwings-crash', 'ottawashooting','sydneysiege']
    # for domain_ID in range(5):
    domain_ID = 0
#选择目标域，与event_list对应
    fewShotCnt = 100
    source_events = [os.path.join(data_dir1, dname)
                     for idx, dname in enumerate(events_list) if idx != domain_ID]
    # source_events = [os.path.join(data_dir1, dname)
    #                  for idx, dname in enumerate(events_list)]
    target_events = [os.path.join(data_dir1, events_list[domain_ID])]
    test_event_name = events_list[domain_ID]
    data_dir2 = r"../../autodl-tmp/data/pheme-rnr-dataset/qwen_gen_from_source/" + test_event_name
    #train_set, labeled_target, val_set, test_set, unlabeled_target
    source_domain, labeled_target, val_set, test_set, unlabeled_target = load_data(
        source_events, target_events, fewShotCnt, unlabeled_ratio=0.3
    )

load len:  1143
load len:  469
load len:  890
load len:  1221
load len:  2079


In [17]:
#加载生成的数据，可以根据训练效果调整正负样本的数量

In [18]:
    print(data_dir2)
    gen_dataset = MetaMCMCDataset()
    gen_dataset.load_data_fast(data_dir2)
    
    gen_target = MetaMCMCDataset()
    gen_target.data = {}
    rumor_count = 0
    non_rumor_count = 0
    actual_non = 0
    actual_rumor = 0
    gen_dataset.data_ID = random.sample(gen_dataset.data_ID, len(gen_dataset.data_ID))
    for i,d_ID in enumerate(gen_dataset.data_ID):
        if gen_dataset.data[d_ID]['label'] == "rumours":
            
            rumor_count += 1
            #控制rumor数量(即label为1的样本)
            if rumor_count % 2 == 0:
                gen_target.data[d_ID] = gen_dataset.data[d_ID]
                actual_rumor += 1
        else:
            non_rumor_count += 1
            #控制non-rumor数量(即label为0的样本)
            if non_rumor_count % 2 == 0:
                gen_target.data[d_ID] = gen_dataset.data[d_ID]
                actual_non += 1
        
    print("rumor:", actual_rumor)
    print("non rumor:", actual_non)
#     random.sample(gen_dataset.data_ID,1500)
    gen_target.dataclear()
    data_list = [unlabeled_target,gen_target]
    new_unlabeled_target = reduce(Merge_data,data_list)

../../autodl-tmp/data/pheme-rnr-dataset/qwen_gen_from_source/charliehebdo
load len:  1328
rumor: 154
non rumor: 510


In [19]:
#开始训练,可供调整的参数主要是DgMSTF_Trainer中的learning_rate（任务学习率），epsilon_ball（扰动范围），G_lr（扰动学习率）
#D_lr（领域判别学习率），dStep（扰动步长）每10步会进行开发集和测试集的验证,主要记录打印出来的Performance中的valid_acc和valid_f1_1
#效果好时截图保存，并记录参数信息

In [26]:
    logDir = f"../../autodl-tmp/pkl/GpDANN/{test_event_name}/"

    print("%s : (dev event)/(test event)/(train event) = %3d/%3d/%3d" % (
        test_event_name, len(val_set), len(test_set), len(source_domain)))
    # print("\n\n===========%s Train===========\n\n" % te.data[te.data_ID[0]]['event'])
    print("\n\n===========%s Out Domain Train===========\n\n" % test_event_name)

    bertPath = r"../../autodl-tmp/bert_en"
    model = obtain_Transformer(bertPath)
    source_domain.initGraph()
    dev_eval = TransformerEvaluator(val_set, batch_size=20)
    te_eval = TransformerEvaluator(test_set, batch_size=20)
    trainer = MCMCBiGCNTrainer(logDir, None, model_rename=False, beam_K=3, batch_dir=f"./aug_dir4")
    if os.path.exists(f"../../autodl-tmp/pkl/GpDANN/{test_event_name}/BiGCN_{test_event_name}.pkl"):
        model.load_model(f"../../autodl-tmp/pkl/GpDANN/{test_event_name}/BiGCN_{test_event_name}.pkl")
    else:
        trainer.fit(model, source_domain, dev_eval, te_eval, batch_size=32, grad_accum_cnt=1, learning_rate=2e-5,
                max_epochs=20,
                model_file=os.path.join(logDir, f'BiGCN_{test_event_name}.pkl'))
        if os.path.exists(f"../../autodl-tmp/pkl/GpDANN/{test_event_name}/BiGCN_{test_event_name}.pkl"):
            model.load_model(f"../../autodl-tmp/pkl/GpDANN/{test_event_name}/BiGCN_{test_event_name}.pkl")
        else:
            model.save_model(f"../../autodl-tmp/pkl/GpDANN/{test_event_name}/BiGCN_{test_event_name}.pkl")    

    #可以考虑修改的参数包括learning_rate,epsolon_ball,G_lr,D_lr,dStep
    trainer = DgMSTF_Trainer(random_seed=10086, log_dir=logDir, suffix=f"{test_event_name}_FS{fewShotCnt}",model_file=f"../../autodl-tmp/pkl/GpDANN/DgMSTF_{test_event_name}_FS{fewShotCnt}.pkl", domain_num=5,class_num=2, temperature=0.05, learning_rate=4e-5, batch_size=24, epsilon_ball=5e-4, Lambda=0.1, G_lr = 5e-4, D_lr=2e-4, valid_every=10, dStep=20) 
    bert_config = BertConfig.from_pretrained(bertPath,num_labels = 2)
    model_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    discriminator = DomainDiscriminator(hidden_size=bert_config.hidden_size,
                                        model_device = model_device,
                                        learningRate=2e-5,
                                        domain_num=5)
    trainer.domain_discriminator = discriminator
    print("being domain discriminate!")
    if os.path.exists(f"../../autodl-tmp/pkl/GpDANN/DomainDiscriminator_{test_event_name}.pkl"):
        trainer.domain_discriminator.load_state_dict(
            torch.load(f"../../autodl-tmp/pkl/GpDANN/DomainDiscriminator_{test_event_name}.pkl")
        )
    else:
        for epoch in range(2):
            trainer.optimizeDiscriminator(model, source_domain, new_unlabeled_target, max_step=500)
        torch.save(trainer.domain_discriminator.state_dict(), f"../../autodl-tmp/pkl/GpDANN/DomainDiscriminator_{test_event_name}.pkl")
    trainer.Training(model, source_domain, new_unlabeled_target, test_set, dev_eval, te_eval, max_iterate=100)

charliehebdo : (dev event)/(test event)/(train event) = 593/1386/3723


===========charliehebdo Out Domain Train===========




3723it [00:00, 296594.31it/s]


being domain discriminate!


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/da-mstf/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/anaconda3/envs/da-mstf/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/ubuntu/da-mstf_rumor/da-mstf_rumor/BaseModel/modeling_bert.py", line 1100, in forward
    encoder_outputs = self.encoder(
  File "/usr/local/anaconda3/envs/da-mstf/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/ubuntu/da-mstf_rumor/da-mstf_rumor/BaseModel/modeling_bert.py", line 691, in forward
    layer_outputs = layer_module(
  File "/usr/local/anaconda3/envs/da-mstf/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    r